In [1]:
import sklearn, gensim, numpy, matplotlib

In [2]:
from sklearn import cluster, datasets

In [3]:
from time import time

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, TfidfTransformer, CountVectorizer

In [5]:
from sklearn.pipeline import make_pipeline

In [6]:
from sklearn.preprocessing import Normalizer

### Setting up some training / testing text

In [7]:
from sklearn.datasets import fetch_20newsgroups

In [8]:
categories = ['alt.atheism','soc.religion.christian','comp.graphics','sci.med']

In [9]:
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

In [10]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [11]:
len(twenty_train.data)

2257

In [12]:
twenty_train.data

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

In [13]:
len(twenty_train.filenames)

2257

In [14]:
print("\n".join(twenty_train.data[2].split("\n")[:15]))

From: djohnson@cs.ucsd.edu (Darin Johnson)
Subject: Re: harrassed at work, could use some prayers
Organization: =CSE Dept., U.C. San Diego
Lines: 63

(Well, I'll email also, but this may apply to other people, so
I'll post also.)

>I've been working at this company for eight years in various
>engineering jobs.  I'm female.  Yesterday I counted and realized that
>on seven different occasions I've been sexually harrassed at this
>company.

>I dreaded coming back to work today.  What if my boss comes in to ask
>me some kind of question...


In [15]:
print(twenty_train.target_names[twenty_train.target[2]])

soc.religion.christian


In [16]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2])

In [17]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


# Extracting Textual Features

In [18]:
count_vect = CountVectorizer()

In [19]:
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [20]:
X_train_counts.shape

(2257, 35788)

In [21]:
X_train_counts.data[0:30]

array([1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [22]:
count_vect.vocabulary_.get(u'algorithm')

4690

In [23]:
twenty_train.data[0:2]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer

In [28]:
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)

In [29]:
X_train_tf = tf_transformer.transform(X_train_counts)

In [30]:
X_train_tf.shape

(2257, 35788)

In [31]:
tfidf_transformer = TfidfTransformer()

In [32]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [33]:
X_train_tfidf.shape

(2257, 35788)

In [38]:
list(zip(list(count_vect.vocabulary_.keys())[15:25], list(X_train_tfidf.data[15:25])))

[('places', 0.034290706362898604),
 ('borrowed', 0.036044147187848298),
 ('bgrahame', 0.11382738609462074),
 ('beseeched', 0.017762318563562172),
 ('19th', 0.088654162537216877),
 ('llegare', 0.065675780431863881),
 ('polarized', 0.016864892977128034),
 ('asteroid', 0.059661620128702707),
 ('citations', 0.037793189755988436),
 ('hvp', 0.043161997007118758)]

### Lots of garbage because unique words and stopwords represent many of these features;
### we'll change that now

In [39]:
# hasher = HashingVectorizer(n_features = 10000, stop_words='english', non_negative=True, norm='l2', binary=False)

In [40]:
# vectorizer = make_pipeline(hasher, TfidfTransformer())

In [41]:
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000, min_df=3, stop_words='english', norm='l2', use_idf=True)

In [42]:
X = vectorizer.fit_transform(twenty_train.data)

In [44]:
list(zip(vectorizer.get_feature_names()[3000:3010], X.data[3000:3010]))

[('documented', 0.021322409721422842),
 ('documents', 0.017788384070372847),
 ('dodell', 0.042009046356661965),
 ('does', 0.018001877759448455),
 ('doesn', 0.029369513528380489),
 ('doesnt', 0.11734500890045807),
 ('dog', 0.023824946920233195),
 ('dogfight', 0.011504688254801885),
 ('dogma', 0.030973551328140295),
 ('dogmatic', 0.18965624177466298)]

# Clustering

In [46]:
from sklearn.cluster import KMeans, hierarchical

In [47]:
from sklearn import metrics

In [48]:
true_k = numpy.unique(twenty_train.target_names).shape[0]

In [49]:
true_k

4

In [50]:
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1, verbose=1)

In [51]:
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X) #km.fit(X_train_tfidf)
print("done in %0.3fs" % (time() - t0))

print("Homogeneity: %0.3f" % metrics.homogeneity_score(twenty_train.target, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(twenty_train.target, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(twenty_train.target, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(twenty_train.target, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, twenty_train.target, sample_size=1000))


Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=4, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=1)
Initialization complete
Iteration  0, inertia 4271.588
Iteration  1, inertia 2183.203
Iteration  2, inertia 2172.106
Iteration  3, inertia 2168.631
Iteration  4, inertia 2168.035
Iteration  5, inertia 2167.858
Iteration  6, inertia 2167.812
Iteration  7, inertia 2167.788
Iteration  8, inertia 2167.763
Iteration  9, inertia 2167.744
Iteration 10, inertia 2167.729
Iteration 11, inertia 2167.726
Iteration 12, inertia 2167.723
Converged at iteration 12: center shift 0.000000e+00 within tolerance 9.809767e-09
done in 1.514s
Homogeneity: 0.455
Completeness: 0.537
V-measure: 0.493
Adjusted Rand-Index: 0.379
Silhouette Coefficient: 0.008


In [52]:
sklearn.metrics.homogeneity_score??

In [53]:
terms = vectorizer.get_feature_names()

In [54]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print('\n')

Top terms per cluster:
Cluster 0:
 pitt
 geb
 banks
 gordon
 cs
 dsl
 shameful
 cadre
 surrender
 n3jxp


Cluster 1:
 god
 people
 jesus
 keith
 think
 believe
 don
 com
 bible
 say


Cluster 2:
 com
 msg
 university
 article
 know
 posting
 like
 people
 don
 host


Cluster 3:
 graphics
 files
 image
 file
 program
 thanks
 university
 3d
 windows
 posting




# Do it with your own corpus

# Topic Modeling with Gensim

In [55]:
from gensim import corpora, models, similarities